# Recursion Limits

In this notebook let's make some tries defining `remaining_steps` inside a custom `state_schema` and seeing if it works for recursion limit control. 

For example build a class with `remanining_steps ` and try to make it halt by itself. If it doesn't, it means that LangGraph doesn't handle it automatically. 

Then after that try and set explicit control flows. I know how to do it in a `Command[Literal["next_possible_node1", "next_possible_node2"]]` fashion, but my doubt is: in my supervisor graph I have no nodes. Maybe I should create it from scratch in order to have that kind of control?

> Here are some sources for this topic:
> - [Graph-API: Impose a recursion limit](https://langchain-ai.github.io/langgraph/how-tos/graph-api/?utm_source=chatgpt.com#impose-a-recursion-limit);
> - ['Medium' post about recursion limits](https://medium.com/@pankajchandravanshi/df371792c8b9);
> - [Adding `remaining_steps` fixed a recursion error problem](https://stackoverflow.com/questions/79446089/langgraph-create-react-agent-with-sqltoolkit-issue-sorry-need-more-steps-to-pr?utm_source=chatgpt.com).

Let's make a very simple example: a custom state with a counter that we increment by $1$ at every step (basically the step counter).

In [ ]:
from typing import Annotated
from langgraph.graph import MessagesState
from langgraph.managed.is_last_step import RemainingSteps
from pydantic import Field

# reducer
def counter_add(value: int):
    return value + 1

# custom state
class CustomState(MessagesState):
    counter: Annotated[int, counter_add] = Field(default=0)
    remaining_steps: RemainingSteps = Field(default=20)

> **Why Use `Field()` in Pydantic (and LangGraph)**
>
> When defining default values in a `pydantic.BaseModel`, use `Field()` to avoid bugs — especially with **mutable types** like `dict`, `list`, and `set`.
>
> #### 🚫 What *not* to do
>
> ```python
> class State(BaseModel):
>     loaded: dict = {}  # ❌ Shared across all instances
> ```
>
> #### ✅ Correct way
>
> ```python
> from pydantic import Field
>
> class State(BaseModel):
>     loaded: dict = Field(default_factory=dict)  # ✅ New dict per instance
> ```
>
> This ensures that each instance gets its own fresh dictionary and prevents side effects across agents in LangGraph.
>
> #### Simple rule of thumb
>
> * Use `Field(default_factory=...)` for mutable types (like `dict`, `list`)
> * Use `Field(default=...)` or direct `=` for immutable types (like `int`, `str`)
>
> #### In LangGraph
>
> LangGraph's custom states often contain fields like `messages`, `loaded`, and `remaining_steps`. Use `Field(...)` to ensure proper initialization and avoid shared state bugs between agent runs.
